# Tools, API & Microservices

Now that we have seen the power of prompts and a look how they come together in a simple agent, lets explore formally a few other concepts.

1. Function calling
2. Tool Calling
3. Introduction to Agents
4. Agents calling tools
5. Agentic Patterns
6. Agents and Microservices

_Each module is typically dependent on the prior modules having been completed successfully_


# Agent

Agents are the heart of complex AI applications. They combine inference, memory, safety, and tool usage into coherent workflows. At its core, an agent follows a sophisticated execution loop that enables multi-step reasoning, tool usage, and safety checks.
Ref: https://llama-stack.readthedocs.io/en/latest/building_applications/agent_execution_loop.html

An Agentic-AI eco system is much larger than the LLM/s which it uses. While LLMs are being used, the agentic structure helps us to automate using those sophisticated prompts that we talked about.  The power of agentic AI is not only in the model, but in the orchestration—how you structure the workflow to get durable, repeatable outcomes without hand-holding.


### Goal-Oriented Looping

- A raw LLM gives one-shot answers. An agent keeps trying, planning multiple steps, checking for errors, adapting.
- Think of it as: “Try → Check → Revise → Retry → Finish” or "Thought → Action → Observation → Repeat → Answer"
- The loop itself enforces discipline and depth.
- Without that structure, the LLM might shortcut the process.

### Memory & Scratchpad

Agents can keep track of:
- What they’ve tried
- What the intermediate results were
- What the user originally wanted
- LLM alone doesn’t track history or outcomes unless explicitly given.

### Tool Use

- Agents can call APIs, browse docs, or query databases. LLM alone hallucinates data. An agent says: “I don’t know—let me look it up.”

### Decomposition

- Agents break big tasks into smaller ones.
- LLMs can do this, but often need a prompt to do so.
- Agents automate that “thinking out loud.”




As we go intot the Agents Module we need a new Python dependancy

In [3]:
%pip install openai-agents==0.0.13


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import openai
import re
import httpx
import os
import requests
import rich
import json
from openai import OpenAI
import requests

# from agents import Agent, ModelSettings, function_tool,Runner

# model = "phi4"
# model = "llama3.2:3b-instruct-fp16"
# model = "phi4"
# model = "qwen3:32b"

api_key = "placeholder" 
model = "mistral-small:latest"
base_url = "http://localhost:11434/v1/"

from rich import print
print("[green] Model setup[/green]")

 Model setup

## Agent calling tools
1. Simply demonstrates an agent using a tool.
1. Look at the brevity of the code compared to doing a function calling all on our own.
1. Play with the question that can be asked to agent to see how it can handle questions that may or may not require the tool

In [36]:
from agents import Agent, ModelSettings, function_tool, Runner, AsyncOpenAI, OpenAIChatCompletionsModel

# Configure the model
# model = OpenAIChatCompletionsModel( 
#     model="qwen3:32b",
#     openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1",api_key = api_key)
# )

model = OpenAIChatCompletionsModel( 
    model=model,
    openai_client=AsyncOpenAI(base_url=base_url, api_key=api_key)
)

In [37]:

@function_tool
def get_weather(latitude:str, longitude:str) ->str:
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

agent = Agent(
    name="Blaster",
    instructions="Answer the question asked very precisely. Please think before answering",
    model= model,
    tools=[get_weather],
)

result = await Runner.run(agent, "which is warmer now: Paris or Manila?")
print(result.final_output)

Based on the data, Manila is warmer now.

Manila's current temperature: 27.1°C

Paris' current temperature: 16.7°C

In [33]:
from rich.pretty import pprint
pprint(result.final_output)

'Manila is warmer now at 27.2°C compared to Paris at 16.9C°'

In [32]:
from rich.pretty import pprint
pprint(result)

RunResult(
│   input='which is warmer now: Paris or Manila?',
│   new_items=[
│   │   ToolCallItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer the question asked very precisely. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7f398189cc50>,
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=None,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None,
│   │   │   │   │   reasoning=None,
│   │   │   │   │   metadata=None,
│   │   │   │   │   store=None,
│   │   │   │   │   include_usage=None,
│   │   │   │   │   extra_query=None,
│   │   │   │   │   extra_body=None,
│   │   │   │   │   extra_headers=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   │   │   │   │   'longitude': {'title': 'Longitude', 'type': 'string'}
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   'required': ['latitude', 'longitude'],
│   │   │   │   │   │   │   'title': 'get_weather_args',
│   │   │   │   │   │   │   'type': 'object',
│   │   │   │   │   │   │   'additionalProperties': False
│   │   │   │   │   │   },
│   │   │   │   │   │   on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f39824127a0>,
│   │   │   │   │   │   strict_json_schema=True
│   │   │   │   │   )
│   │   │   │   ],
│   │   │   │   mcp_servers=[],
│   │   │   │   mcp_config={},
│   │   │   │   input_guardrails=[],
│   │   │   │   output_guardrails=[],
│   │   │   │   output_type=None,
│   │   │   │   hooks=None,
│   │   │   │   tool_use_behavior='run_llm_again',
│   │   │   │   reset_tool_choice=True
│   │   │   ),
│   │   │   raw_item=ResponseFunctionToolCall(
│   │   │   │   arguments='{"latitude":"14.5995","longitude":"120.9842"}',
│   │   │   │   call_id='call_jojssoy9',
│   │   │   │   name='get_weather',
│   │   │   │   type='function_call',
│   │   │   │   id='__fake_id__',
│   │   │   │   status=None
│   │   │   ),
│   │   │   type='tool_call_item'
│   │   ),
│   │   ToolCallItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer the question asked very precisely. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7f398189cc50>,
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=None,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None,
│   │   │   │   │   reasoning=None,
│   │   │   │   │   metadata=None,
│   │   │   │   │   store=None,
│   │   │   │   │   include_usage=None,
│   │   │   │   │   extra_query=None,
│   │   │   │   │   extra_body=None,
│   │   │   │   │   extra_headers=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   

## Optional - explore the Agent's "working"

Printing the result with `pprint` will allow you to explore the Agent's working

# Tools
- We are definining tools with an adornment here.
- MCP Servers are formalizing this much more and everyone is adopting this.
- This is covered in another lab.

# Agentic Patterns


We explore below 3 agentic paradigms which are widely used

1. Agents collaborating with each other to improve the quality of the output
   ![Collaboration Pattern](resources/images/agent_collaborate.png)
   <img src="resources/images/agent_collaborate.png" alt="description" width="300" height="200">
3. Agents routing traffic to the correct agent
   ![pattern-1](resources/images/agent_supervisor_pattern.png)  
   ![pattern-2](resources/images/agent_hierarchical.png) 
5. Agents running a workflow
   ![Workflow Pattern](resources/images/agent_plan_execute.png)

There are other agentic patterns as well which we do not cover here. But these basic concepts should help adopting other patterns much simpler.

_The graphics have been used from [langraph tutorial](https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials)_

## Agents Collaborating
1. Simply demonstrates an agent reviewing the work of another agent - much like a human being.
1. This is one of the primary reasons while the agents can help increase accuracty of the answer and smaller models using agents can outperform larger models without agents.
1. This pattern can be used in lots of scenarios.

In [38]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

"""
This example shows the agents collaborating or one agent reveiwing the work of another and giving feedback. 
The first agent generates an outline for a story.
The second agent judges the outline and provides feedback. 
We loop until the judge is satisfied with the outline.
"""

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
    model= model,
)


@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]


evaluator = Agent(
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
    ),
    model= model,
    output_type=EvaluationFeedback,
)

print("Finshed story generator setup")

Finshed story generator setup

In [39]:
msg = input("What kind of story would you like to hear? ")
input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_outline: str | None = None

with trace("Collaboration"):
        i = 0
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")

            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break
            if i == 2:
                print("Maximum number of iterations exceeded, exiting.")
                break
            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})
            i += 1

print(f"Final story outline: {latest_outline}")



What kind of story would you like to hear?  dark fiction


Story outline generated

Evaluator score: pass

Story outline is good enough, exiting.

Final story outline: **Short Story Outline: "Whispers of the Abyss"**

**Genre:** Dark Fiction

**Title:** Whispers of the Abyss
---

### Title: "Whispers of the Abyss"

- **Protagonist**: Emma, a reclusive writer who recently lost her husband.
- **Setting**: An old, isolated cottage in a dense forest during winter months.

- **Hook**:
  - "After all, it should be easy to write about one's lost love"
    Emma decides to spend the coming winter by herself in order to get through writers block

  -

#### Act I: Descent into Madness
1. **Inciting Incident**: Emma discovers an eerie, abandoned diary in the attic and begins writing with it.
2.  **Conflict**: Odd occurrences begin to happen as Emma spends more time writing her story. The cottage starts to
feel alive, with unseen voices and noises echoing around
3. **Catalyst**: At a particular moment while working late into the night she starts hearing her name being called

#### Act II: The Abyss Deepens
-  "Whispers are often only heard when you've been staring down at an abyss for so long"
    - There is an underlying sense of dread as the reader will want to know where these voices are starting to come
from.
4. **Conflict Escalates**: Emma starts seeing ghosts and hearing whispers constantly. She finds evidence that the 
diary was used by a past inhabitant who may not have left this place willingly

5. **Climax**:
    -  Her characters turn very sinister and start speaking to her.

-  The forest around her has started to change and as she attempts to leave her surroundings become more ominous
   Emma realizes that the voices are not just figments of her imagination: they come from a hidden truth behind the
cottage walls. She uncovers an evil presence that controls the area, intent on making sure things never end up 
leaving its influence

#### Act III: Confrontation and Resolution


6. **Dénouement**:
  She realizes everything has been leading to ending all at once.

Conclusion:
In Emma’s mind there are two possibilities- either the story was trying to warn against getting too attached to her
words or it was a final letter of goodbye from her husband.
The last line is her making the choice "she decided to set off to get lost in some words, but not before writing 
this one thing down: 'you chose me and I’ll write my way to you'""

## Agents routing
1. Simply demonstrates an agent routing work to other agents.
1. This is a very common agentic pattern.
1. Ask the question in German and see what happens! In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.

In [31]:

import uuid

#from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

# from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

"""
This example shows the handoffs/routing pattern. The triage agent receives the first message, and
then hands off to the appropriate agent based on the language of the request. Responses are
streamed to the user.
"""

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
    model = model,
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
    model = model,
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English. Answer the question you recieved.",
    model = model,
)

# experiment by removing  from the instructions the sentence below.
#So answer in English even if you understand the language that is being used.
#And then ask say (German) : Wie geht es dir
know_all_agent = Agent(
    name="know_all_agent",
    instructions="You only speak English. So answer in English even if you understand the language that is being used. \
        State that you do not understand the user question and ask them to repeat it one of the languages you understand. \
        Those languages are English, French and Spanish .",
    model = model,
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request. If you do not know what to do, hand if off to know_all.",
    handoffs=[french_agent, spanish_agent, english_agent, know_all_agent],
    model = model,
)



In [ ]:

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

with trace("Router"):
    story_outline_result = await Runner.run(triage_agent,inputs)
    #uncomment this to see the details
    #pprint(story_outline_result)
    print("--------------------------")
    print(story_outline_result.final_output)


--------------------------
<think>
Okay, the user wrote "monami Hastings!" in French. Let me check if that's a name or a phrase.

"Monami" isn't a standard French word. Maybe they meant "mon ami" which is "my friend". But "Hastings" is likely the last name from the TV show "Bridgerton". So, the user might be referring to the character Hastings from the show. 

I need to acknowledge the greeting. Since "monami" could be a misspelling, I'll respond in a friendly way, assuming they're addressing me as Hastings. I should keep the reply in French and offer help. 

Make sure to keep the response simple and welcoming, confirming understanding and offering assistance. Avoid any complex structures since the user might be learning.
</think>

Bonjour! Je suis Hastings. Comment puis-je vous aider aujourd'hui?


## Agents Deterministic Workflow
1. Simply demonstrates agents calling other agents to complete a well defined workflow.
1. This is a very common agentic pattern.
1. This pattern or its variants can be put to lot of practical use and it could be combined with the collaborative pattern.

In [8]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

"""
This example shows how different agents are used to compelete a deterministic workflow.
In this case it is: 
planner agent -> writer agent -> editor agent 
Given an essay topic, the essay moves through these stages to finally produce an output.
"""

@dataclass
class Planner:
    body: str

@dataclass
class Writer:
    body: str

@dataclass
class Editor:
    body: str



planner_agent = Agent(
    name="planner_agent",
    instructions=(
        "Take a user's theme/topic request."
        "Create a brief outline of the essay with points that need to be covered."
        "Make sure that references are given to actual source materials."
    ),
    model= model,
    output_type=Planner,
)



writer_agent = Agent(
    name="writer_agent",
    instructions=(
        "Take the outline given in the input."
        "Expands it into a complete essay."
        "Give adequate references and make sure things are not made up!"
    ),
    model= model,
    output_type=Writer,
)

editor_agent = Agent(
    name="editor_agent",
    instructions=(
        "You Review the draft given in the input."
        "Polish the language, fixes inconsistencies, and improve the flow."
        "And make sure it is logical coherent."
        "Give adequate references and make sure things are not made up!"
        "Return the final story to the user."
    ),
    model= model,
    output_type=Editor,
)



In [9]:
msg = input("Hi! I am AI Researcher. Give me any topic and I will write a well researched essay about it. ")
inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

with trace("Workflow"):
    print("----------Planner Output----------")
    planner_result = await Runner.run(planner_agent,inputs)
    print(planner_result.final_output.body)
    planner_output: Planner = planner_result.final_output
    print("----------Writer Output----------")
    writer_result = await Runner.run(writer_agent,planner_output.body)
    print(writer_result.final_output.body)
    writer_output: Writer = writer_result.final_output
    print("----------Editor Output----------")
    editor_result = await Runner.run(editor_agent,writer_output.body)
    print(editor_result.final_output.body)

Hi! I am AI Researcher. Give me any topic and I will write a well researched essay about it.  History of agents in AI


----------Planner Output----------
Sure, I can help with developing an essay outline on the history of agents in AI. An agent, in the context of artificial intelligence, is any entity that perceives its environment through sensors and acts upon it through actuators to achieve a goal effectively. Here's an outline you could follow, along with some points and references:

### Essay Title: The Evolution of Agents in Artificial Intelligence

#### I. Introduction
   - Definition of an intelligent agent.
   - Importance of agents in the field of AI.
   - Brief overview of the historical progression of AI agents (from rule-based systems to modern AI).
   - References: [Russell & Norvig, 2003] [Nilsson, 2009]

#### II. Early Foundations of AI Agents
   - The concept of thinking machines: Alan Turing's "Computing Machinery and Intelligence" (1950) introducing the Turing Test.
   - Early rule-based systems like Newell and Simon's General Problem Solver (GPS) from 1959.
   - The emergence of logi

----------Planner Output----------
Sure!  Creating an essay on the concept of agents in artificial intelligence through time involves tracing the evolution from early theoretical ideas to modern-day autonomous systems. Here’s a brief outline you can follow, including the key points and references to source materials.
----------Writer Output----------
Okay, the user wants me to take the outline they provided and turn it into a complete essay about the concept of agents in AI over time. They also want references and accurate information. Let me start by understanding the outline they mentioned. The outline includes tracing the evolution from early theories to modern autonomous systems. They probably expect sections on different time periods or key developments, such as classical AI, reactive agents, utility-based agents, learning agents, multi-agent systems, and maybe ethical or future considerations. They mentioned John McCarthy, Russell and Norvig, and maybe some other researchers like

# Microservices

There are several meaningful similarities between LLM-based AI agents and microservices:

## Similarities
#### Specialized functionality: 
Both are designed to handle specific tasks or domains. Microservices focus on particular business capabilities, while AI agents can be specialized for specific types of interactions or knowledge domains.
### Independent operation: 
Both can operate autonomously within their defined scope. Once configured, they can process requests without requiring constant supervision.
### Communication patterns: 
Both typically communicate via messages/APIs. Microservices use REST/gRPC/messaging protocols, while AI agents receive prompts and return responses through APIs.
### Composability: 
Both can be combined to build larger systems. Microservices can be orchestrated to create complex applications; similarly, multiple AI agents can work together in a workflow.
### Statelessness vs. statefulness: 
Basic implementations of both can be stateless, but more sophisticated versions maintain state. The Agent class you showed maintains conversation history, similar to how some microservices maintain session state.
### Scaling considerations: 
Both face similar operational challenges around scaling, monitoring, and versioning.

## Key differences:

### Implementation: 
Microservices are traditional code with deterministic logic, while LLM agents use probabilistic models. MCP Servers which expose tools to be used by Agents could be totally traditional code with deterministic logic.
### Predictability: 
Microservices have more predictable outputs for given inputs, while LLM responses can vary.


# AFTERWORD
Agents are an extremely powerful construct in the field of Generative AI:
1. You can achieve complex tasks designing appropriate agents and tools and driving interaction between the different agents.
1. There are known ways by which we can improve accuracy of the output. Much like human beings help check one another's work, agents can do the same.
1. External data retrieval and queries are carried out through the tools.
1. If agent processing needs to be vetted, make sure humans are used (human-in-the-loop) to are used to vet the agent output before it moves to the next step. Really, this is no different to how we operate in our real life with human beings - we have review and approval processes etc.